<center><h1> BrainHack School Project 2020 </h1></center>
<center><h3> Stephanie Alley </h3></center>
<center><h6> PhD student in Biomedical Engineering </h6></center>
<center><h6> Polytechnique Montreal </h6></center>


# Background
<h4> Pre-processing pipeline for prostate mpMRI</h4>

<center><img src="prostate.png" style="margin-top: 20px" /></center>

 <center><img src="crying.png" /></center>

# What I wanted to learn

<table>
    <tr>
        <td style="width:50%">
                 <h2>Nilearn</h2>
                <h2>Git/GitHub</h2>
                <h2>Datalad</h2>
                <h2>Jupyter Notebook</h2>
                <h2>Plotly</h2>
                <h2>Binder</h2>
        </td>
        <td style="width:50%">
            <center><img src="tools.png" style="" /></center>
        </td>
    </tr>
</table>

# Project

#### Assess effects of processing on age prediction using rs-fMRI

* Atlas choice
* Confound choice
* Machine learning (Support Vector Regression)

# rs-fMRI processing

<center><img src="pipeline.png" style="margin-bottom: 500px" /></center>


# rs-fMRI processing
<center><img src="pipeline_detail.png" style="margin-bottom: 500px" /></center>

# Atlas choice

<center><img src="atlases.png" style="padding-top:100px" /></center>

# Confound choice

<center><img src="confound_choice.png" style="padding-top:100px" /></center>

# Machine learning: Nilearn
### Repeated cross-validation

<center><img src="ML_CV.jpeg" style="margin-bottom: 500px" /></center>

In [7]:
%%html
<style>
.mytext {
    font-size: 50px;
}
</style>

In [8]:
from plotly import graph_objects
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table
from ipywidgets import widgets
import scipy.io as sio
import numpy as np
import pandas as pd

def calculate(x, y):
    X = sm.add_constant(x)
    res = sm.OLS(y, X).fit()
    st, data, ss2 = summary_table(res, alpha=0.05)
    preds = pd.DataFrame.from_records(data, columns=[s.replace('\n', ' ') for s in ss2])
    preds['displ'] = x
    preds = preds.sort_values(by='displ')
    return preds

out = sio.loadmat('output.mat')
x=np.squeeze(out['BASC']['All'][0,0][0,0][0])
y=np.squeeze(out['BASC']['All'][0,0][0,0][1])

preds = calculate(np.squeeze(out['BASC']['All'][0,0][0,0][0]),np.squeeze(out['BASC']['All'][0,0][0,0][1]))


textbox = widgets.Dropdown(
    description='<b style="font-size: 15px">Atlas:   </b>',
    value='BASC',
    options=['BASC', 'AAL', 'CRAD', 'MSDL'],
    layout={'font-size': '50px'}
)

textbox.add_class("mytext")

slider = widgets.SelectionSlider(
    options=['All', 'CSF', 'WM', 'Global', 'Motion'],
    value='All',
    description='<b style="font-size: 15px">Confounds</b>',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout={'width': '100%', 'font-size': '50px'}
)

p1 = graph_objects.Scatter(**{
    'mode' : 'markers',
    'x' : x,
    'y' : y,
    'name' : 'Points'
})
p2 = graph_objects.Scatter({
    'mode' : 'lines',
    'x' : preds['displ'],
    'y' : preds['Predicted Value'],
    'name' : 'Regression',
})
#Add a lower bound for the confidence interval, white
p3 = graph_objects.Scatter({
    'mode' : 'lines',
    'x' : preds['displ'],
    'y' : preds['Mean ci 95% low'],
    'name' : 'Lower 95% CI',
    'showlegend' : False,
    'line' : {
        'color' : 'white'
    }
})
# Upper bound for the confidence band, transparent but with fill
p4 = graph_objects.Scatter( {
    'type' : 'scatter',
    'mode' : 'lines',
    'x' : preds['displ'],
    'y' : preds['Mean ci 95% upp'],
    'name' : '95% CI',
    'fill' : 'tonexty',
    'line' : {
        'color' : 'white'
    },
    'fillcolor' : 'rgba(255, 127, 14, 0.3)'
})
g = graph_objects.FigureWidget(data=[p1, p2, p3, p4],
                    layout={
                        'title' : {
                            'text': "Age prediction on test set for each atlas",
                            'y': 0.9,
                            'x': 0.5,
                            'xanchor': 'center',
                            'yanchor': 'top'
                        },
                        'xaxis' : {
                            'title' : 'Predicted Log Age'
                        },
                        'yaxis' : {
                            'title' : 'Log Age'
                        }
                    })
def response(change):
    x = np.squeeze(out[textbox.value][slider.value][0,0][0,0][0])
    y = np.squeeze(out[textbox.value][slider.value][0,0][0,0][1])
    preds = calculate(x,y)
    g.data[0].x = x
    g.data[0].y = y
    g.data[1].x = preds['displ']
    g.data[1].y = preds['Predicted Value']
    g.data[2].x = preds['displ']
    g.data[2].y = preds['Mean ci 95% low']
    g.data[3].x = preds['displ']
    g.data[3].y = preds['Mean ci 95% upp']
        
textbox.observe(response, names="value")
slider.observe(response, names="value")
container = widgets.VBox([textbox,
              g])
#widgets.VBox([container, slider])

In [9]:
widgets.VBox([textbox,
              g])
widgets.VBox([container, slider])

# Results

<center><img src="results_datalad.png" style="margin-bottom: 500px; width: 150%;" /></center>

# Takeaways

* REPRODUCIBILITY IS POSSIBLE!!!
* DataLad keeps track of data manipulation
* TIER structure for organization
* Share code using GitHub
* Nilearn for machine learning